In [1]:
!pip install gym
!pip install box2d-py

In [2]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Conv1D, MaxPooling1D, Flatten, Dropout
from keras.optimizers import Adam
from keras import backend as K
print(K.tensorflow_backend._get_available_gpus())
from IPython.display import display, clear_output
from scipy.special import softmax
import random
from sklearn import preprocessing
import matplotlib.pyplot as plt
import time

import os

Using TensorFlow backend.


[]


In [0]:
state_size = 8
action_size = 4

In [0]:
# Deep Q-learning Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen = 3000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9995
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.walk = deque(maxlen = 900)
        
    def target_model_update(self):
        self.target_model.set_weights(self.model.get_weights())
        
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        
        model.add(Dense(64, input_dim=state_size, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=0.00015))
        
        model.summary()
        return model
    
    def remember(self, state, action, reward, next_state, done):
#         print(state)
#         print(reward)
        self.memory.append((state, action, reward, next_state, done))
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            chose = np.random.randint(0,4)
            return chose
        
        act_values = 0
        act_values = self.model.predict(state)
        chose = np.argmax(act_values)
#         print(act_values, chose)
        if chose == 0:
            return chose
        if chose == 1:
            return chose
        if chose == 2:
            return chose
        if chose == 3:
            return chose
        return act_values
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        minibatch = np.array(minibatch)
        not_done_indices = np.where(minibatch[:, 4] == False)
        y = np.copy(minibatch[:, 2])
        predict_sprime = self.model.predict(np.vstack(minibatch[:, 3]))
        predict_sprime_target = self.target_model.predict(np.vstack(minibatch[:, 3]))
        
        # If minibatch contains any non-terminal states, use separate update rule for those states
        if len(not_done_indices[0]) > 0:
            predict_sprime = self.model.predict(np.vstack(minibatch[:, 3]))
            predict_sprime_target = self.target_model.predict(np.vstack(minibatch[:, 3]))
            
            # Non-terminal update rule ###?
            y[not_done_indices] += np.multiply(self.gamma, \
                        predict_sprime_target[not_done_indices, \
                        np.argmax(predict_sprime[not_done_indices, :][0], axis=1)][0])
            
        actions = np.array(minibatch[:, 1], dtype=int)
        y_target = self.model.predict(np.vstack(minibatch[:, 0]))
        y_target[range(batch_size), actions] = y
        self.model.fit(np.vstack(minibatch[:, 0]), y_target, verbose = 0)

                
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        
        random.shuffle(self.memory)

    
    def save_model(self):
        self.model.save('./checkp.h5')
        np.save("game_mem", self.memory)

    def load_model(self):
        self.model.load_weights('./checkp.h5')
        self.memory = np.load("game_mem.npy", allow_pickle=True)
        self.memory = deque(self.memory)

In [0]:
def count_consec_300(x):
    cnt = 0
    for i in range(len(x)):
        if x[i] >= 300:
            cnt += 1
        else:
            cnt = 0
    return cnt

def reward_greater(rew, no_improve):
    if rew < 0:
        no_improve += 1
    else:
        no_improve = 0
    return no_improve

In [0]:
env = gym.make('LunarLander-v2')
agent = DQNAgent(state_size, action_size)
# agent.load_model()
done = False
batch_size = 32
game_history = deque(maxlen = 100)

# game_history = np.load("game_history.npy")
explore = True
t_steps = 0
for e in range(10000):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
#     print(state.shape)
    total_reward = 0
    prev_reward = 0
    start = time.time()
    for timee in range(450):
        # env.render()
#         if time % 5 == 0:
#             print(time, end=', ')
        

        
        action = agent.act(state)
        
        next_state, reward, done, _ = env.step(action)
#         print(action, end='')
#         if done :
#             done = True
#             reward = -100
        

        next_state = np.reshape(next_state, [1, state_size])

        agent.remember(state, action, reward, next_state, done)
        state = next_state
        
        if done:
            agent.target_model_update()
            break
        
        prev_reward = reward
        total_reward += reward
        if len(agent.memory) > 1000:
            agent.replay(batch_size)
            explore = False
#         print("diff:", reward - prev_reward,"prev:", prev_reward, " current:", reward, "total reward:", total_reward)


    game_history.append(total_reward)
#     if explore == False:
    print("round:{} score: {} mean:{} len of mem:{} spent:{} epsilon {}".format(
        e, total_reward, np.mean(game_history), len(agent.memory), time.time() - start, agent.epsilon))
        

    done = False


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                576       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 4,996
Trainable params: 4,996
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                576       
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
__